# 前処理

## 環境構築

In [1]:
# Notebook初期設定
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm

import sqlalchemy as sa
import sqlite3
import re
import shutil

In [3]:
DIR_IN = '../data/scatter_scrape'
DIR_OUT = '../data/scatter_preprocess'

In [4]:
FN_UM = 'umamusume.csv'

FN_DB = 'keiba.db'
TN_RACE = 'race'
TN_RES = 'result'

In [5]:
def make_df_by_query(path_db, query):
    """path_dbにqueryを投げた結果のdfを取得"""
    conn = sqlite3.connect(path_db, isolation_level=None)
    cur = conn.cursor()
    df = pd.read_sql(query, conn)
    return df

## 前提

`DIR_IN`に下記のようなファイルが存在することを前提としている．
- `keiba.db`：netkeibaからレース結果を取得
    - `race`テーブル：レースの概要情報
    - `result`テーブル：着順等の情報
    - netkeibaからのスクレイピングに関してはネット上に優れた解説記事が散見されるため，詳細は省略
- `umamusume.csv`：ウマ娘の適性情報
    - `./scatter_scrape.ipynb`にてスクレイピング

### `keiba.db`

In [6]:
path_db = os.path.join(DIR_IN, FN_DB)

In [7]:
# raceテーブルの雰囲気をつかむ
df_race_head = \
    make_df_by_query(path_db, 'SELECT * FROM race LIMIT 5;')
## スッキリ表示したいので転置する
df_race_head.head().T

,0,1,2,3,4
index,0,1,2,3,4
race_id,198601010101,198601010102,198601010103,198601010104,198601010105
date,1986-06-07,1986-06-07,1986-06-07,1986-06-07,1986-06-07
place,札幌,札幌,札幌,札幌,札幌
race_name,4歳以上300万下,3歳新馬,4歳未勝利,4歳未勝利,4歳未勝利
distance,1500,1000,1200,1800,1500
dart,True,True,True,True,True
dart_cond,良,良,良,良,良
turf,False,False,False,False,False
turf_cond,None,None,None,None,None


In [8]:
# resultテーブルの雰囲気を掴む
df_result_head = \
    make_df_by_query(path_db, 'SELECT * FROM result LIMIT 5;')
## スッキリ表示したいので転置する
df_result_head.head().T

,0,1,2,3,4
race_id,198601010101,198601010101,198601010101,198601010101,198601010101
arrival_order,1,2,3,4,5
frame_no,2,3,5,6,7
horse_no,2,3,5,6,7
horse_name,ワクセイ,マツタカラオー,カンキョウヘルス,スズタカエース,クリヤーパーマン
horse_id,1983502460,1983502412,1983501389,1982502619,1983500572
horse_sex,Male,Male,Male,Male,Male
horse_age,4,4,4,5,4
horse_weight,468.0,430.0,460.0,456.0,432.0
horse_weight_change,0.0,4.0,-4.0,14.0,-6.0


### `umamusume.csv`

In [9]:
df_um = pd.read_csv(os.path.join(DIR_IN, FN_UM))

In [10]:
## スッキリ表示したいので転置する
df_um.head().T

,0,1,2,3,4
um_name,アグネスデジタル,エアグルーヴ（花嫁）,エイシンフラッシュ,エルコンドルパサー（新衣装）,オグリキャップ
horse_name,アグネスデジタル,エアグルーヴ,エイシンフラッシュ,エルコンドルパサー,オグリキャップ
turf_index,A,A,A,A,A
dart_index,A,G,G,B,B
short_index,F,C,G,F,E
mile_index,A,B,F,A,A
intermediate_index,A,A,A,A,A
long_index,G,E,A,B,B
nige_index,G,D,G,E,F
senko_index,A,A,B,A,A


## raceテーブルの更新

In [41]:
def get_title(race_name):
    """race_nameのからタイトル情報を抽出"""
    # 空白文字を除外
    title = re.sub('\s$', '', race_name)
    # 第n回
    title = re.sub('第\d+回', '', title)
    # グレード：(Gn)，(J.Gn)
    title = re.sub('\((J.G|G)\d+\)', '', title)
    # リステッド競争：(L), 重賞：(G)
    title = re.sub('\((L|G)\)', '', title)
    # (n)
    title = re.sub('\(\d+\)', '', title)
    # 第n戦
    title = re.sub('第\d+戦', '', title)
    # 末尾の数字
    title = re.sub('\d$', '', title)
    # 西暦
    title = re.sub('19[8-9][0-9]|20[0-2][0-9]', '', title)
    # 略式の西暦
    title = re.sub('’([8-9][0-9]|[0-2][0-9])', '', title)
    return title

In [42]:
def get_grade(race_name):
    """race_nameからグレード情報を抽出"""
    r = re.search('(\((J.G|G)\d+\)|\((L|G)\))', race_name)
    if r:
        grade = r.group().replace('(', '').replace(')', '')
    else:
        grade = None
    return grade

### DBのコピー

In [43]:
path_from = os.path.join(DIR_IN, FN_DB)
path_to = os.path.join(DIR_OUT, FN_DB)

In [44]:
# コピー先のDBが初期化されるので注意
shutil.copy(path_from, path_to)

'../data/scatter_preprocess/keiba.db'

### raceテーブルの更新

In [45]:
# raceテーブルの全データを取得
path_db = os.path.join(DIR_OUT, FN_DB)
df_race = make_df_by_query(path_db, 'SELECT * FROM race;')

In [46]:
# 集計用にカラムを追加
df_race['title'] = df_race['race_name'].apply(
    lambda x: get_title(x))
df_race['grade'] = df_race['race_name'].apply(
    lambda x: get_grade(x))

In [47]:
df_race.head().T

,0,1,2,3,4
index,0,1,2,3,4
race_id,198601010101,198601010102,198601010103,198601010104,198601010105
date,1986-06-07,1986-06-07,1986-06-07,1986-06-07,1986-06-07
place,札幌,札幌,札幌,札幌,札幌
race_name,4歳以上300万下,3歳新馬,4歳未勝利,4歳未勝利,4歳未勝利
distance,1500,1000,1200,1800,1500
dart,True,True,True,True,True
dart_cond,良,良,良,良,良
turf,False,False,False,False,False
turf_cond,None,None,None,None,None


In [48]:
# 各グレードのレース数を集計してみる
df_race['grade'].value_counts()

G3      2197
G2      1139
G1       703
G        137
L        125
J.G3     110
J.G2      66
J.G1      44
Name: grade, dtype: int64

In [49]:
df_race.shape[0]

119898

In [50]:
# 全体に対する比率
df_race['grade'].value_counts() / df_race.shape[0]

G3      0.018324
G2      0.009500
G1      0.005863
G       0.001143
L       0.001043
J.G3    0.000917
J.G2    0.000550
J.G1    0.000367
Name: grade, dtype: float64

In [51]:
# to_sqlを普通に実行するとすごく遅いため，method='multi'を選択したいがエラーが出る．
# どうやらPandasのバグっぽい（https://github.com/pandas-dev/pandas/issues/29921）
# 諦めてsqlalchemy経由で接続する
# この場合closeは不要？よくわからない
engine = sa.create_engine(
    f'sqlite:///{path_db}', echo=False)
df_race.to_sql(
    TN_RACE, engine, if_exists='replace',
    method='multi', chunksize=5000)

## プロット用csvの準備

- raceとresultをジョインし，重賞のみにフィルタした`all_res.csv`を作成
- 育成ウマ娘の適性データを保存した`umamusume.csv`はそのままコピー

### `all_res.csv`

In [52]:
def query_results_and_races_by_grade(grade):
    """グレードでレース結果を集計するクエリ"""
    q = f'''
        SELECT *
        FROM (
            SELECT * FROM race 
            WHERE grade = '{grade}'
            AND steeple = 'False'
        ) AS race_g
        INNER JOIN result
        ON race_g.race_id = result.race_id;
    '''
    return q

In [53]:
def add_average_speed_to_df(df):
    """平均の速さを計算して追加"""
    df_new = df.copy()
    df_new = df_new[~df_new['seconds_total'].isna()].\
        reset_index(drop=True)
    df_new = df_new[~df_new['seconds_3f'].isna()].\
        reset_index(drop=True)
    df_new['speed_total'] = \
        df_new['distance'] / df_new['seconds_total'] * 60 * 60 / 1000
    df_new['speed_3f'] = \
        600 / df_new['seconds_3f'] * 60 * 60 / 1000
    return df_new

In [54]:
def get_distance_class(distance):
    """ウマ娘における距離区分を返す
    https://altema.jp/umamusume/kyoritekisei
    """
    if distance < 1600:
        return 'short'
    elif distance < 2000:
        return 'mile'
    elif distance < 2500:
        return 'intermediate'
    elif distance >= 2500:
        return 'long'
    else:
        return None

In [55]:
def add_distance_class_to_df(df):
    """距離区分をdfに追加"""
    df_new = df.copy()
    df_new['distance_class'] = \
        df_new['distance'].apply(
            lambda x: get_distance_class(x))
    return df_new

In [56]:
def make_df_by_grade_from_db(path_db, grade):
    """dbからgradeを元にpd.DataFrameを作成を作成"""
    q = query_results_and_races_by_grade(grade)
    df = make_df_by_query(path_db, q)
    df = add_average_speed_to_df(df)
    df = add_distance_class_to_df(df)
    return df

In [57]:
# 抽出対象とするグレード
## G.J\dは障害競走のグレードのため除外
grades = ['G1', 'G2', 'G3', 'G', 'L']

In [58]:
df_all = pd.DataFrame()
for g in tqdm(grades):
    df = make_df_by_grade_from_db(path_db, g)
    df_all = pd.concat([df_all, df], ignore_index=True)

  0%|          | 0/5 [00:00<?, ?it/s]

In [59]:
# 中身はこんな感じ
## スッキリ表示したいので転置する
df_all.head().T

,0,1,2,3,4
level_0,1038,1038,1038,1038,1038
index,1038,1038,1038,1038,1038
race_id,198605020811,198605020811,198605020811,198605020811,198605020811
date,1986-05-11,1986-05-11,1986-05-11,1986-05-11,1986-05-11
place,東京,東京,東京,東京,東京
race_name,第36回安田記念(G1),第36回安田記念(G1),第36回安田記念(G1),第36回安田記念(G1),第36回安田記念(G1)
distance,1600,1600,1600,1600,1600
dart,False,False,False,False,False
dart_cond,None,None,None,None,None
turf,True,True,True,True,True


In [60]:
# level_0とindexが邪魔なので削除
df_all = df_all.drop(columns=['index', 'level_0'])

In [61]:
df_all.to_csv(os.path.join(DIR_OUT, 'all_res.csv'), index=False)

### `umamusume.csv`のコピー

特に処理は必要ない．単純にコピーするだけ

In [62]:
path_from = os.path.join(DIR_IN, FN_UM)
path_to = os.path.join(DIR_OUT, FN_UM)

In [63]:
shutil.copy(path_from, path_to)

'../data/scatter_preprocess/umamusume.csv'

## （予備分析）重賞レースに出馬できる馬の割合

平地競走に出馬経験のある馬のうち，どの程度の割合が各グレードのレースに出場できるかを集計する

In [73]:
path_db = os.path.join(DIR_OUT, FN_DB)

In [74]:
# 平地競走に出場経験のある競走馬の数
query = '''
SELECT count(DISTINCT(horse_id))
FROM race INNER JOIN result
ON race.race_id = result.race_id
WHERE steeple = 'False';
'''
df = make_df_by_query(path_db, query)

In [75]:
n_horse = df.values[0][0]

In [81]:
n_horse

149610

In [77]:
# 重賞（G1，G2，G3，G，L）の全結果
df_all = pd.read_csv(os.path.join(DIR_OUT, 'all_res.csv'))

In [79]:
# 各グレードに出場経験のある馬の数
df_all.groupby('grade')['horse_id'].nunique()

grade
G       290
G1     4981
G2     6966
G3    11356
L       857
Name: horse_id, dtype: int64

In [80]:
# 平地競走に出場経験のある競走馬に対する割合
df_all.groupby('grade')['horse_id'].nunique() / n_horse

grade
G     0.001938
G1    0.033293
G2    0.046561
G3    0.075904
L     0.005728
Name: horse_id, dtype: float64